In [2]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from collections import Counter

In [3]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('train.csv').fillna(' ')
test = pd.read_csv('test.csv').fillna(' ')

In [5]:
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [4]:
all_2 = all_text.str.lower().str.split('\s+')

In [5]:
all_3 = all_2.values.flatten().tolist()

In [6]:
al = []
for i in all_3:
    al.extend(i)

In [7]:
al2 = Counter(al)

In [8]:
def get_key(d, value):
    for k, v in d.items():
        if v == value:
            return k

get_key(al2, max(al2.values()))

'the'

In [10]:
from sklearn.feature_extraction import stop_words
from scipy.sparse import hstack

submission = pd.DataFrame.from_dict({'id': test['id']})

stopwords = stop_words.ENGLISH_STOP_WORDS
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words=stopwords,
    ngram_range=(1, 1),
    max_features=20000)
word_vectorizer.fit(all_text)

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 4),
    max_features=30000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)


train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

classifier = LogisticRegression(solver='sag', n_jobs = -1)

scores= []

for class_name in class_names:
    train_target = train[class_name]

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, scoring='roc_auc'))
    
    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]
    
    print('CV score for class {} is {}'.format(class_name, cv_score))
    scores.append(cv_score)

print('Total score is {}'.format(np.mean(scores)))

submission.to_csv('submission.csv', index=False)

CV score for class toxic is 0.9791090894854154
CV score for class severe_toxic is 0.9889568125465499
CV score for class obscene is 0.9906791775804517
CV score for class threat is 0.9895653635672611
CV score for class insult is 0.9831328999254482
CV score for class identity_hate is 0.9832334666679431
Total score is 0.9857794682955116


In [11]:
submission = pd.DataFrame.from_dict({'id': test['id']})

In [12]:
for class_name in class_names:
    train_target = train[class_name]

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, scoring='roc_auc'))
    
    classifier.fit(train_features, train_target)
    
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]
    scores.append(cv_score)

In [13]:
submission.to_csv('submission.csv', index=False)